In [2]:
from quantumML.rest import MWRester

In [4]:
b=MWRester()


[1.7320956495978288e-09,
 1.7320956495978274e-09,
 1.6814056067898388e-09,
 1.6814056067898446e-09,
 1.681405606789842e-09,
 1.6814056067898408e-09]

# Queries
- **band_gap_range**  
Range of bandgaps in *eV* as a two valued list eg. [1.2, 3.1]  
- **Formation_energy_range**  
Range of formation energies in *meV* as two values list eg. [50, 150]
- **elements**  
List of elements contained in entry eg. ['Na', 'Cl']  
note: this will return all entries that contain Na and Cl as well as any other element 
- **space_group_number**  
integer of space group number eg. 5
- **dimention**  
integer of dimension of material eg. 2
- **crystal_system**  
crystal system as a string eg. 'Tetragonal'  


In [12]:
help(b.get_calculation)

Help on method get_calculation in module quantumML.rest:

get_calculation(band_gap_range=None, formation_energy_range=None, elements=[], space_group_number=None, dimension=None, crystal_system=None, name=None) method of quantumML.rest.MWRester instance
    Method to that queries materialsweb database and returns a list of dictionaries of calculations that
    fit the querries parameters. Additionally
    Parameters:
        band_gap_range (list): List of band gap range e.g. [min, max]
        formation_energy_range (list): List of formation energy range e.g. [min, max]
        elements (list): List of str of elements
        space_group_number (int): space group number
        dimension (int): dimension as int e.g. 1 2 3
        crystal_system (str): name of crystal system
        name (str): name of material e.g. MoS2
    Returns:
        results: List of results matching quire parameters



In [5]:
d = b.get_calculation(band_gap_range=[1,1.5], formation_energy_range=[124,150])
for d in b.results:
    #print(d.keys())
    if len(d['composition'].split('/')[-2].replace('%',''))< 8:
        print(d['composition'].split('/')[-2].replace('%','')+'\t\t\t BandGap = ' + \
              str(round(d['band_gap'],4))+' eV\t Formation Eneergy = ' + str(d['formation_energy'])+ ' meV')
    elif len(d['composition'].split('/')[-2].replace('%',''))< 16:
        print(d['composition'].split('/')[-2].replace('%','')+'\t\t BandGap = ' + \
              str(round(d['band_gap'],4))+' eV\t Formation Eneergy = ' + str(d['formation_energy'])+ ' meV')
    else:
        print(d['composition'].split('/')[-2].replace('%','')+'\t BandGap = ' + \
              str(round(d['band_gap'],4))+' eV\t Formation Eneergy = ' + str(d['formation_energy'])+ ' meV')


<class 'quantumML.rest.MWRester'>
Pd120S2			 BandGap = 1.1746 eV	 Formation Eneergy = 148.0 meV
Ge120Se1		 BandGap = 1.2107 eV	 Formation Eneergy = 133.0 meV
Pd120Se2		 BandGap = 1.3964 eV	 Formation Eneergy = 166.0 meV
I220Pt1			 BandGap = 1.4215 eV	 Formation Eneergy = 135.0 meV
I220Pd1			 BandGap = 1.1512 eV	 Formation Eneergy = 133.0 meV
Br320Cr1		 BandGap = 1.4714 eV	 Formation Eneergy = 742.0 meV
Cr120H120O2		 BandGap = 1.0072 eV	 Formation Eneergy = 179.0 meV
Mo120O3			 BandGap = 1.1381 eV	 Formation Eneergy = 136.0 meV


# Write files

In [11]:
help(b.write)
#help(b.write_all)

Help on method write in module quantumML.rest:

write(index=0) method of quantumML.rest.MWRester instance
    Writes INCAR, KPOINTS, POSCAR of entry to current directory
    
    Parameters:
        index (int): index of entry to write files for

Help on method write_all in module quantumML.rest:

write_all() method of quantumML.rest.MWRester instance
    Creates a directory named by composition for every entry in results. Then, Writes INCAR, KPOINTS,
    POSCAR of entry to respective directory



In [6]:
import os 
os.mkdir('example')
%cd example
print(type(b))
b.write_all()
%ls
%cd ..

FileExistsError: [Errno 17] File exists: 'example'

In [ ]:
%cd ~/dev/mwinterface
%pwd

# Machine Learning

In [13]:
#Convert to list of pymatgen structures

c = b.as_pymatgen_struc()

[1.7320956495978288e-09,
 1.7320956495978274e-09,
 1.6814056067898388e-09,
 1.6814056067898446e-09,
 1.681405606789842e-09,
 1.6814056067898408e-09]

In [15]:
MWRester().get_symmetry_functions_g2(c[0])
b.get_soap(b.results[0])

TypeError: can only concatenate str (not "dict") to str

In [17]:
!ls
!mkdir temp

build  LICENSE	  quantumML.egg-info  requirments.txt  tutorial.ipynb
dist   quantumML  README.md	      setup.py	       Untitled.ipynb


In [18]:
!cd temp
b.prep_ml_formation_energy(b.results[-1])

In [19]:
!ls

0.energy   12.energy  15.energy  18.energy  3.energy  6.energy	9.energy
0.poscar   12.poscar  15.poscar  18.poscar  3.poscar  6.poscar	9.poscar
10.energy  13.energy  16.energy  1.energy   4.energy  7.energy	OsZICAR
10.poscar  13.poscar  16.poscar  1.poscar   4.poscar  7.poscar	XdATCAR
11.energy  14.energy  17.energy  2.energy   5.energy  8.energy
11.poscar  14.poscar  17.poscar  2.poscar   5.poscar  8.poscar


In [95]:
def get_soap(calculation, rcut=7, nmax=5, lmax=8, fmt='MW'):
    if fmt == 'MW':
        urlp = 'http://2dmaterialsweb.org/' + calculation['path'][22:] + '/POSCAR'
        file = urllib.request.urlopen(urlp)
        file = file.read().decode("utf-8")
        file = io.StringIO(file)
    elif fmt == 'poscar':
        file = calculation
    ml=vasp.read_vasp(file)
    species=['Cd','Te']
    periodic_soap = SOAP(
    periodic=True,
    species=np.unique(ml.get_atomic_numbers()),
    rcut=rcut,
    nmax=nmax,
    lmax=lmax,
    rbf='gto',
    sigma=0.125,
    average=True
    )
    soap = periodic_soap.create(ml)
    #soap = 1
    return soap

In [96]:
get_soap( b.results[0])

array([[ 3.23715732e-02,  4.29601129e-03,  3.62247764e-03,
        -1.82748225e-03,  6.71045855e-03,  5.72189747e-04,
         4.98028065e-04, -2.51956750e-04,  8.52994621e-04,
         5.49888762e-04, -2.83340196e-04,  4.37105395e-04,
         1.46175546e-04, -2.07636374e-04,  2.07245979e-03,
         4.84641851e-06, -1.27033127e-05,  7.06361743e-06,
         1.84710243e-05,  4.33171999e-06,  3.33478856e-05,
        -1.84206419e-05, -4.91428691e-05, -1.06311736e-05,
         1.28466208e-05,  2.68671120e-05, -5.54062217e-06,
         8.16366737e-05, -1.23714744e-06,  8.77736602e-05,
         3.66675499e-06, -1.02916611e-05,  1.22379970e-06,
         2.65821218e-05,  1.30838416e-05,  2.95985719e-05,
        -2.10575607e-07, -8.84286492e-05, -2.97435090e-05,
         3.44540713e-05, -6.00897511e-05, -5.35784675e-05,
         4.63482313e-04, -1.40571256e-05,  5.42705297e-04,
         9.64503761e-06, -2.70058135e-05,  2.48923261e-05,
         3.16124606e-05, -9.66213793e-06,  7.56741429e-0